#Cài đặt PySpark

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar -xvf spark-3.2.1-bin-hadoop3.2.tgz

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

In [ ]:
!python -m pip install --upgrade pip
!pip install -q findspark
!pip install pyspark

     |████████████████████████████████| 2.1 MB 5.3 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 KB 13.1 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=b31e5ce2b6e4c7959042858c1750fdc101956cf751b40853f232547caa258ce6
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
import pyspark
# Creating a SparkSession in Python
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local")\
          .appName("StructureAPI")\
          .config("spark.some.config.option", "some-value")\
          .getOrCreate()

sc = spark.sparkContext

#Dữ liệu nhỏ

##Tải dữ liệu nhỏ

In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip ml-latest-small.zip

--2022-04-02 08:04:19--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  --.-KB/s    in 0.1s    

2022-04-02 08:04:20 (8.21 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [ ]:
!ls

ml-latest-small      sample_data		spark-3.2.1-bin-hadoop3.2.tgz
ml-latest-small.zip  spark-3.2.1-bin-hadoop3.2


##Nhập file dữ liệu nhỏ

In [ ]:
small_datasets_path = 'ml-latest-small'

In [ ]:
small_ratings_file = os.path.join(small_datasets_path, 'ratings.csv')
small_ratings_raw_data = sc.textFile(small_ratings_file) 
small_ratings_raw_data_header = small_ratings_raw_data.take(1)[0] 
small_ratings_data = small_ratings_raw_data.filter(lambda line: line!=small_ratings_raw_data_header).map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache() 


In [ ]:
print(small_ratings_data.take(10))

[('1', '1', '4.0'), ('1', '3', '4.0'), ('1', '6', '4.0'), ('1', '47', '5.0'), ('1', '50', '5.0'), ('1', '70', '3.0'), ('1', '101', '5.0'), ('1', '110', '4.0'), ('1', '151', '5.0'), ('1', '157', '5.0')]


##Số liệu cho mô hình ALS

###Chia nhỏ tập dữ liệu

In [ ]:
training_RDD, validation_RDD, test_RDD = small_ratings_data.randomSplit([6, 2, 2], seed=0)
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1])) 
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

###Thông số cho ALS train

In [ ]:
from pyspark.mllib.recommendation import ALS
import math

seed = 5
iterations = 10
regularization_parameter = 0.1
#ranks = [4, 8, 12]
#errors = [0, 0, 0]
#err = 0
#tolerance = 0.02

##Thực hiện train

In [ ]:
model_test = ALS.train(training_RDD, rank=8, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
predictions = model_test.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = test_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)

##RMSE metric

In [ ]:
result_RMSE = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
print('For testing data the RMSE is %s' % (result_RMSE))

For testing data the RMSE is 0.9100699365514616


##Hit rate metric

In [ ]:
from collections import defaultdict
import numpy as np

In [ ]:
#Lọc top N
#Mặc định top 10 với mức rating là 3
def GetTopN(predictions, n=10, minimumRating=3.0):
  topN = defaultdict(list)
  for userID, movieID, actualRating, estimatedRating in predictions:
    if (estimatedRating >= minimumRating):
      topN[int(userID)].append((int(movieID), estimatedRating))
  for userID, ratings in topN.items():
      ratings.sort(key=lambda x: x[1], reverse=True)
      topN[int(userID)] = ratings[:n]
  return topN

In [ ]:
def HitRate(topNPredicted, leftOutPredictions):
  hits = 0
  total = 0
  # For each left-out rating
  for leftOut in leftOutPredictions:
      userID = leftOut[0]
      leftOutMovieID = leftOut[1]
      # Is it in the predicted top 10 for this user?
      hit = False
      for movieID, predictedRating in topNPredicted[int(userID)]:
        if (int(leftOutMovieID) == int(movieID)):
          hit = True
          break
      if (hit) :
        hits += 1
      total += 1
  # Compute overall precision
  return hits/total

In [ ]:
preds = []
for i in np.array(rates_and_preds.collect()):
  preds.append(np.array(i).flatten())
tests = test_for_predict_RDD.map(lambda x: (int(x[0]), int(x[1])))

result_HitRate = HitRate(GetTopN(preds), np.array(tests.collect()))

print('For testing data the HR is %s' % (result_HitRate))

For testing data the HR is 0.23046181172291297


##NDCG metric


In [ ]:
def NDCG(rel_true, rel_pred, p=0, form="linear"):
    rel_true = np.sort(rel_true)[::-1]
    p = min(len(rel_true), min(len(rel_pred), p))
    discount = 1 / (np.log2(np.arange(p) + 2))

    if form == "linear":
        idcg = np.sum(rel_true[:p] * discount)
        dcg = np.sum(rel_pred[:p] * discount)
    elif form == "exponential" or form == "exp":
        idcg = np.sum([2**x - 1 for x in rel_true[:p]] * discount)
        dcg = np.sum([2**x - 1 for x in rel_pred[:p]] * discount)
    else:
        raise ValueError("Only supported for two formula, 'linear' or 'exp'")

    return dcg / idcg

In [ ]:
rating_true = rates_and_preds.map(lambda x: x[1][0])
rating_pred = rates_and_preds.map(lambda x: x[1][1])
result_NDCG = NDCG(rating_true.collect(), rating_pred.collect(), 4)

print('For testing data the HR is %s' % (result_NDCG))

For testing data the HR is 0.8291152648363814


#Dữ liệu lớn

##Tải dữ liệu lớn

In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip ml-1m.zip
!ls

--2022-04-02 08:13:59--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  33.1MB/s    in 0.2s    

2022-04-02 08:13:59 (33.1 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         
ml-1m		 ml-latest-small.zip	    spark-3.2.1-bin-hadoop3.2.tgz
ml-1m.zip	 sample_data
ml-latest-small  spark-3.2.1-bin-hadoop3.2


##Nhập dữ liệu lớn

In [ ]:
big_datasets_path = 'ml-1m'
big_ratings_file = os.path.join(big_datasets_path, 'ratings.dat')
big_ratings_raw_data = sc.textFile(big_ratings_file) 
#big_ratings_raw_data_header = small_ratings_raw_data.take(1)[0] 
#big_ratings_data = small_ratings_raw_data.filter(lambda line: line!=small_ratings_raw_data_header).map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache() 
big_ratings_data = big_ratings_raw_data.map(lambda line: line.split("::")).map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache() 


In [ ]:
big_ratings_data.take(10)

[('1', '1193', '5'),
 ('1', '661', '3'),
 ('1', '914', '3'),
 ('1', '3408', '4'),
 ('1', '2355', '5'),
 ('1', '1197', '3'),
 ('1', '1287', '5'),
 ('1', '2804', '5'),
 ('1', '594', '4'),
 ('1', '919', '4')]

##Nhập số liệu ALS

###Chia nhỏ tập dữ liệu

In [ ]:
training_bRDD, test_bRDD = big_ratings_data.randomSplit([6,4], seed=0)
test_for_predict_bRDD = test_bRDD.map(lambda x: (x[0], x[1]))

###Thông số train ALS train

In [ ]:
from pyspark.mllib.recommendation import ALS
import math

seed = 5
iterations = 10
regularization_parameter = 0.1
#ranks = [4, 8, 12]
#errors = [0, 0, 0]
#err = 0
#tolerance = 0.02

##Thực hiện train

In [ ]:
Bmodel_test = ALS.train(training_bRDD, rank=4, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
predictions = big_ratings_data.predictAll(test_for_predict_bRDD).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = test_bRDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)

##RMSE metrix

In [ ]:
result_RMSE = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
print('For testing data the RMSE is %s' % (result_RMSE))

For testing data the RMSE is 0.9301901571026362


##Hit rate metric

In [ ]:
from collections import defaultdict
import numpy as np

In [ ]:
#Lọc top N
#Mặc định top 10 với mức rating là 3
def GetTopN(predictions, n=10, minimumRating=3.0):
  topN = defaultdict(list)
  for userID, movieID, actualRating, estimatedRating in predictions:
    if (estimatedRating >= minimumRating):
      topN[int(userID)].append((int(movieID), estimatedRating))
  for userID, ratings in topN.items():
      ratings.sort(key=lambda x: x[1], reverse=True)
      topN[int(userID)] = ratings[:n]
  return topN

In [ ]:
def HitRate(topNPredicted, leftOutPredictions):
  hits = 0
  total = 0
  # For each left-out rating
  for leftOut in leftOutPredictions:
      userID = leftOut[0]
      leftOutMovieID = leftOut[1]
      # Is it in the predicted top 10 for this user?
      hit = False
      for movieID, predictedRating in topNPredicted[int(userID)]:
        if (int(leftOutMovieID) == int(movieID)):
          hit = True
          break
      if (hit) :
        hits += 1
      total += 1
  # Compute overall precision
  return hits/total

In [ ]:
preds = []
for i in np.array(rates_and_preds.collect()):
  preds.append(np.array(i).flatten())
tests = test_for_predict_bRDD.map(lambda x: (int(x[0]), int(x[1])))

result_HitRate = HitRate(GetTopN(preds), np.array(tests.collect()))

print('For testing data the HR is %s' % (result_HitRate))

For testing data the HR is 0.23046181172291297


##NDCG metric


In [ ]:
def NDCG(rel_true, rel_pred, p=0, form="linear"):
    rel_true = np.sort(rel_true)[::-1]
    p = min(len(rel_true), min(len(rel_pred), p))
    discount = 1 / (np.log2(np.arange(p) + 2))

    if form == "linear":
        idcg = np.sum(rel_true[:p] * discount)
        dcg = np.sum(rel_pred[:p] * discount)
    elif form == "exponential" or form == "exp":
        idcg = np.sum([2**x - 1 for x in rel_true[:p]] * discount)
        dcg = np.sum([2**x - 1 for x in rel_pred[:p]] * discount)
    else:
        raise ValueError("Only supported for two formula, 'linear' or 'exp'")

    return dcg / idcg

In [ ]:
rating_true = rates_and_preds.map(lambda x: x[1][0])
rating_pred = rates_and_preds.map(lambda x: x[1][1])
result_NDCG = NDCG(rating_true.collect(), rating_pred.collect(), 4)

print('For testing data the HR is %s' % (result_NDCG))

For testing data the HR is 0.8291152648363814
